In [ ]:
from sentence_transformers import SentenceTransformer
import torch

In [ ]:
model = SentenceTransformer("hiieu/halong_embedding")

In [ ]:
query = "Bóng đá có lợi ích gì cho sức khỏe?"

In [ ]:
query_embedding = model.encode([query])

In [ ]:
import json

In [ ]:
with open(r'C:\WorkStation\UIT\projectCVforTK\BookRetrieval\data\product_injected_categories.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
len(data)

In [ ]:
from src.engine.text_embedding import TextEmbeddingGenerator

In [ ]:
text_embedding = TextEmbeddingGenerator()

In [ ]:
embedding = await text_embedding.generate_text_embedding("hello world")

In [8]:
import chromadb

In [9]:
client = chromadb.PersistentClient(path="./chromadb")

In [10]:
collection = client.get_or_create_collection(
    name="image_chroma_db",
    metadata={"hnsw:space": "cosine"}
)

In [ ]:
collection.count()

In [ ]:
for item in data:
    text = f"Tên sách: {item['Name']}\n" + f"Nội dung sách: {item['Description']}"
    embedding = await text_embedding.generate_text_embedding(text)
    collection.add(
        embeddings=[embedding],
        documents=[text],
        metadatas=[{'id': str(item['Id']), 'name': item['Name'], 'description': item['Description']}],
        ids=[str(item['Id'])],
    )

In [ ]:
a = collection.get(
    ids=["1"]
)

In [ ]:
print(a['metadatas'])

In [ ]:
import os

In [ ]:
root_dir = r'C:\WorkStation\UIT\projectCVforTK\BookRetrieval\data\images-1-1100'  # Replace with your folder path
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')  # Add more if needed

In [5]:
import torch
import base64
from io import BytesIO
import torchvision.transforms as T
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dinov2_vits14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14_reg')

In [ ]:
transform_image = T.Compose(
    [
        T.ToTensor(),
        T.Resize(244),
        T.CenterCrop(224),
        T.Normalize([0.5], [0.5])
    ]
)

In [ ]:
def load_image(base64_image: str) -> torch.Tensor:
    img_data = base64.b64decode(base64_image)
    img = Image.open(BytesIO(img_data)).convert("RGB")

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

In [ ]:
def get_embedding(base64_image: str):
    transformed_img = load_image(base64_image)
    cuda_transformed_img = transformed_img.to(device)

    embeddings = dinov2_vits14_reg(cuda_transformed_img)

    return embeddings[0].cpu().detach().numpy()

In [4]:
def image_to_base64(image_path):
    with open(image_path, 'rb') as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

In [ ]:
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.lower().endswith(image_extensions):
            full_path = os.path.join(subdir, file)
            base64_image = image_to_base64(full_path)
            image_embedding = get_embedding(base64_image)
            collection.add(
                embeddings=[image_embedding],
                metadatas=[{'product_id': os.path.basename(subdir)[5:], 'image_id': os.path.basename(file)[:-4]}],
                ids=[os.path.basename(file)[:-4]],
            )

In [6]:
base64_image = image_to_base64(r"C:\WorkStation\UIT\projectCVforTK\BookRetrieval\data\images-1-1100\prod_1\img_1_015.jpeg")

In [11]:
results = collection.query(
    query_embeddings=[image_embedding],
    n_results=10
)

NameError: name 'image_embedding' is not defined

In [ ]:
metadatas = results.get("metadatas", [])
if not metadatas:
    print("Failed to retrieve by image. Returning an empty list.")

unique_product_ids = {item["product_id"]
                        for item in metadatas[0] if "product_id" in item}
a = list(unique_product_ids)

In [12]:
from src.dependencies.service_dependency import get_service

c:\Users\Lenovo\anaconda3\envs\UIT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: hiieu/halong_embedding
Using cache found in C:\Users\Lenovo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Lenovo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Lenovo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Lenovo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\blo

In [13]:
service = get_service()

In [14]:
result = await service.search_service.search_by_image_embedding(base64_image)

In [15]:
result

['1',
 '147',
 '236',
 '106',
 '153',
 '300',
 '288',
 '289',
 '298',
 '234',
 '237',
 '283',
 '193',
 '198',
 '276',
 '206',
 '302',
 '215',
 '313',
 '929',
 '290',
 '964',
 '916',
 '743',
 '178',
 '203',
 '238',
 '110',
 '192',
 '918',
 '711',
 '997',
 '970',
 '231',
 '733',
 '1036',
 '1069',
 '687',
 '191',
 '329',
 '1071',
 '1043',
 '891',
 '995',
 '910',
 '633',
 '499',
 '648',
 '1041',
 '351',
 '315',
 '898',
 '915',
 '185',
 '547',
 '1063',
 '974',
 '314',
 '631',
 '363',
 '628',
 '235',
 '717',
 '1012',
 '212',
 '296',
 '957',
 '3',
 '1009',
 '640',
 '912',
 '1070',
 '274',
 '497',
 '176',
 '316',
 '921',
 '965',
 '1082',
 '506',
 '295',
 '310',
 '949',
 '83',
 '244',
 '549',
 '639',
 '555',
 '818',
 '285',
 '597',
 '308',
 '933',
 '195',
 '981',
 '222',
 '293',
 '741',
 '1089',
 '979',
 '531',
 '649',
 '1023',
 '1075',
 '993',
 '1049',
 '676',
 '284',
 '328',
 '638',
 '913',
 '261',
 '281',
 '533',
 '527',
 '271',
 '841',
 '739',
 '38',
 '52',
 '210',
 '1095',
 '1002',
 '368',